In [ ]:
import numpy as np
import pandas as pfd
import mediapipe as mp
import time
import cv2
import math
from ctypes import cast,POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities,IAudioEndpointVolume
wCam , hCam = 640,480
mpHands=mp.solutions.hands # it is used to find the landmarks of the hand in given images
hands=mpHands.Hands() #it is used to call the hand function for detecting the hand in image 
mpDraw = mp.solutions.drawing_utils
cTime,pTime=0,0
cap=cv2.VideoCapture(0)
cap.set(3,wCam)
cap.set(4,hCam)
devices = AudioUtilities.GetSpeakers()
interface=devices.Activate(IAudioEndpointVolume._iid_,CLSCTX_ALL,None)
volume=cast(interface,POINTER(IAudioEndpointVolume))
volume.SetMasterVolumeLevel(0,None)
volRange=volume.GetVolumeRange()
minVol=volRange[0]
maxVol=volRange[1]
while True:
    sucess,img1 =  cap.read()
    imgRGB = cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)  # as cv2 genrate the image in BGR format to convert it into RGB we are using the function cvtColor 
    results=hands.process(imgRGB) # results is the converted image of the hands point into RGB color
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
                mpDraw.draw_landmarks(img1,handLms,mpHands.HAND_CONNECTIONS)
    img=img1
    lmList1=[]
    if results.multi_hand_landmarks:
        myHand = results.multi_hand_landmarks[0]
        for id,lm in enumerate(handLms.landmark):
            h,w,c = img.shape
            cx,cy =int(lm.x*w),int(lm.y*h)
            lmList1.append([id,cx,cy])
    lmList=eval(str(lmList1))
    if len(lmList) != 0:
        x1,y1=lmList[4][1],lmList[4][2]
        x2,y2=lmList[8][1],lmList[8][2]
        cx,cy=(x1+x2)//2,(y1+y2)//2
        cv2.circle(img,(x1,y1),15,(225,0,225),cv2.FILLED)
        cv2.circle(img,(x2,y2),15,(225,0,225),cv2.FILLED)
        cv2.line(img,(x1,y1),(x2,y2),(225,0,225),3)
        cv2.circle(img,(cx,cy),15,(225,0,225),cv2.FILLED)
        length = math.hypot(x2-x1,y2-y1)
        vol=np.interp(length,[0,100],[minVol,maxVol])
        volume.SetMasterVolumeLevel(vol,None)
    cTime=time.time()
    fps=1/(cTime-pTime)
    pTime=cTime
    cv2.putText(img,f'FPS:{int(fps)}',(40,70),cv2.FONT_HERSHEY_COMPLEX,1 , (225,0,225),3)
    cv2.imshow("Img",img)
    cv2.waitKey(1)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break 
cap.release() 
cv2.destroyAllWindows() 
